# PROJECT 2 JUPYTER NOTEBOOK

#### Katie Cason, section: Thursday 2-3:30

### Starter Code:

In [1]:
messages = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

In [2]:
import json
import pprint

In [3]:
p = pprint.PrettyPrinter(indent=1)

In [4]:
f = open("assessment-attempts-20180128-121051-nested.json","r")


In [5]:
s = f.read()
json_data = json.loads(s)
f.close()
len(json_data)

3280

In [6]:
# this will pretty print the json in alphabetic order which may or may not match the file order
p.pprint(json_data[0])

{'base_exam_id': '37f0a30a-7464-11e6-aa92-a8667f27e5dc',
 'certification': 'false',
 'exam_name': 'Normal Forms and All That Jazz Master Class',
 'keen_created_at': '1516717442.735266',
 'keen_id': '5a6745820eb8ab00016be1f1',
 'keen_timestamp': '1516717442.735266',
 'max_attempts': '1.0',
 'sequences': {'attempt': 1,
               'counts': {'all_correct': False,
                          'correct': 2,
                          'incomplete': 1,
                          'incorrect': 1,
                          'submitted': 4,
                          'total': 4,
                          'unanswered': 0},
               'id': '5b28a462-7a3b-42e0-b508-09f3906d1703',
               'questions': [{'id': '7a2ed6d3-f492-49b3-b8aa-d080a8aad986',
                              'options': [{'at': '2018-01-23T14:23:24.670Z',
                                           'checked': True,
                                           'correct': True,
                                           'id': '

In [7]:
def recursive_walk_json_object(j, level):
    """recursively walk through a json object to explore the structure
       dictionaries will be put in alphabetic order to match the pretty print above"""
    
    level += 1
    
    if type(j) is dict:
        dict_2_list = list(j.keys())
        dict_2_list.sort()
        for k in dict_2_list:
            print("   " * level + "L" + str(level), k)
            recursive_walk_json_object(j[k], level)
    
    elif type(j) is list:
        for (i, l) in enumerate(j):
            print("  " * level + "  [" + str(i) + "]")
            recursive_walk_json_object(l, level)
            
    else:
        print("   " * level + " value:", j)

In [8]:
recursive_walk_json_object(json_data[0], -1)

L0 base_exam_id
    value: 37f0a30a-7464-11e6-aa92-a8667f27e5dc
L0 certification
    value: false
L0 exam_name
    value: Normal Forms and All That Jazz Master Class
L0 keen_created_at
    value: 1516717442.735266
L0 keen_id
    value: 5a6745820eb8ab00016be1f1
L0 keen_timestamp
    value: 1516717442.735266
L0 max_attempts
    value: 1.0
L0 sequences
   L1 attempt
       value: 1
   L1 counts
      L2 all_correct
          value: False
      L2 correct
          value: 2
      L2 incomplete
          value: 1
      L2 incorrect
          value: 1
      L2 submitted
          value: 4
      L2 total
          value: 4
      L2 unanswered
          value: 0
   L1 id
       value: 5b28a462-7a3b-42e0-b508-09f3906d1703
   L1 questions
      [0]
         L3 id
             value: 7a2ed6d3-f492-49b3-b8aa-d080a8aad986
         L3 options
          [0]
               L5 at
                   value: 2018-01-23T14:23:24.670Z
               L5 checked
                   value: True
               L

In [9]:
raw_assessments = spark.read.format("kafka").option("kafka.bootstrap.servers", "kafka:29092").option("subscribe","assessments").option("startingOffsets", "earliest").option("endingOffsets", "latest").load() 

raw_assessments.cache()

assessments = raw_assessments.select(raw_assessments.value.cast('string'))

import json

from pyspark.sql import Row

extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()

extracted_assessments.registerTempTable('assessments')

spark.sql("select keen_id from assessments limit 10").show()

spark.sql("select keen_timestamp, sequences.questions[0].user_incomplete from assessments limit 10").show()

+--------------------+
|             keen_id|
+--------------------+
|5a6745820eb8ab000...|
|5a674541ab6b0a000...|
|5a67999d3ed3e3000...|
|5a6799694fc7c7000...|
|5a6791e824fccd000...|
|5a67a0b6852c2a000...|
|5a67b627cc80e6000...|
|5a67ac8cb0a5f4000...|
|5a67a9ba060087000...|
|5a67ac54411aed000...|
+--------------------+

+------------------+-------------------------------------------------------+
|    keen_timestamp|sequences[questions] AS `questions`[0][user_incomplete]|
+------------------+-------------------------------------------------------+
| 1516717442.735266|                                                   true|
| 1516717377.639827|                                                  false|
| 1516738973.653394|                                                  false|
|1516738921.1137421|                                                  false|
| 1516737000.212122|                                                  false|
| 1516740790.309757|                                         

In [10]:
spark.sql("select sequences.abc123 from assessments limit 10").show()

+------+
|abc123|
+------+
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
|  null|
+------+



In [11]:
# does NOT work!
#spark.sql("select sequence.id from assessments limit 10").show()

In [12]:
def my_lambda_sequences_id(x):
    raw_dict = json.loads(x.value)
    my_dict = {"keen_id" : raw_dict["keen_id"], "sequences_id" : raw_dict["sequences"]["id"]}
    return Row(**my_dict)

my_sequences = assessments.rdd.map(my_lambda_sequences_id).toDF()

my_sequences.registerTempTable('sequences')

spark.sql("select sequences_id from sequences limit 10").show()

spark.sql("select a.keen_id, a.keen_timestamp, s.sequences_id from assessments a join sequences s on a.keen_id = s.keen_id limit 10").show()


+--------------------+
|        sequences_id|
+--------------------+
|5b28a462-7a3b-42e...|
|5b28a462-7a3b-42e...|
|b370a3aa-bf9e-4c1...|
|b370a3aa-bf9e-4c1...|
|04a192c1-4f5c-4ac...|
|e7110aed-0d08-4cb...|
|5251db24-2a6e-424...|
|066b5326-e547-4da...|
|8ac691f8-8c1a-403...|
|066b5326-e547-4da...|
+--------------------+

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|        sequences_id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|8ac691f8-8c1a-403...|
|5a26ee9cbf5ce1000...|1512500892.4166169|9bd87823-4508-4e0...|
|5a29dcac74b662000...|1512692908.8423469|e7110aed-0d08-4cb...|
|5a2fdab0eabeda000...|1513085616.2275269|cd800e92-afc3-447...|
|5a30105020e9d4000...|1513099344.8624721|8ac691f8-8c1a-403...|
|5a3a6fc3f0a100000...| 1513779139.354213|e7110aed-0d08-4cb...|
|5a4e17fe08a892000...|1515067390.1336551|9abd5b51-6bd8-11e...|
|5a4f3c69cc6444000...| 1515142249.858722|083844

In [13]:
def my_lambda_questions(x):
    raw_dict = json.loads(x.value)
    my_list = []
    my_count = 0
    for l in raw_dict["sequences"]["questions"]:
        my_count += 1
        my_dict = {"keen_id" : raw_dict["keen_id"], "my_count" : my_count, "id" : l["id"]}
        my_list.append(Row(**my_dict))
    return my_list

my_questions = assessments.rdd.flatMap(my_lambda_questions).toDF()

my_questions.registerTempTable('questions')

spark.sql("select id, my_count from questions limit 10").show()

spark.sql("select q.keen_id, a.keen_timestamp, q.id from assessments a join questions q on a.keen_id = q.keen_id limit 10").show()

+--------------------+--------+
|                  id|my_count|
+--------------------+--------+
|7a2ed6d3-f492-49b...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|95194331-ac43-454...|       4|
|95194331-ac43-454...|       1|
|bbed4358-999d-446...|       2|
|e6ad8644-96b1-461...|       3|
|7a2ed6d3-f492-49b...|       4|
|b9ff2e88-cf9d-4bd...|       1|
|bec23e7b-4870-49f...|       2|
+--------------------+--------+

+--------------------+------------------+--------------------+
|             keen_id|    keen_timestamp|                  id|
+--------------------+------------------+--------------------+
|5a17a67efa1257000...|1511499390.3836269|803fc93f-7eb2-412...|
|5a17a67efa1257000...|1511499390.3836269|f3cb88cc-5b79-41b...|
|5a17a67efa1257000...|1511499390.3836269|32fe7d8d-6d89-4db...|
|5a17a67efa1257000...|1511499390.3836269|5c34cf19-8cfd-4f5...|
|5a26ee9cbf5ce1000...|1512500892.4166169|0603e6f4-c3f9-4c2...|
|5a26ee9cbf5ce1000...|1512500892.4166169|26a06b

In [14]:
def my_lambda_correct_total(x):
    
    raw_dict = json.loads(x.value)
    my_list = []
    
    if "sequences" in raw_dict:
        
        if "counts" in raw_dict["sequences"]:
            
            if "correct" in raw_dict["sequences"]["counts"] and "total" in raw_dict["sequences"]["counts"]:
                    
                my_dict = {"correct": raw_dict["sequences"]["counts"]["correct"], 
                           "total": raw_dict["sequences"]["counts"]["total"]}
                my_list.append(Row(**my_dict))
    
    return my_list

my_correct_total = assessments.rdd.flatMap(my_lambda_correct_total).toDF()

my_correct_total.registerTempTable('ct')

spark.sql("select * from ct limit 10").show()

spark.sql("select correct / total as score from ct limit 10").show()

spark.sql("select avg(correct / total)*100 as avg_score from ct limit 10").show()

spark.sql("select stddev(correct / total) as standard_deviation from ct limit 10").show()

+-------+-----+
|correct|total|
+-------+-----+
|      2|    4|
|      1|    4|
|      3|    4|
|      2|    4|
|      3|    4|
|      5|    5|
|      1|    1|
|      5|    5|
|      4|    4|
|      0|    5|
+-------+-----+

+-----+
|score|
+-----+
|  0.5|
| 0.25|
| 0.75|
|  0.5|
| 0.75|
|  1.0|
|  1.0|
|  1.0|
|  1.0|
|  0.0|
+-----+

+-----------------+
|        avg_score|
+-----------------+
|62.65699745547047|
+-----------------+

+-------------------+
| standard_deviation|
+-------------------+
|0.31086692286170553|
+-------------------+



In [15]:
raw_assessments.write.mode('overwrite').parquet("/tmp/raw_assessments")

### Business Questions:

Describe your assumptions, your thinking, what the parts of the pipeline do. What is given? What do you set up yourself? Talk about the data. What issues do you find with the data? Do you find ways to solve it? How? If not describe what the problem is.

#### 1. How many people started an exam at 12 PM?

In [23]:
spark.sql('select count(user_exam_id) as started_at_12 from assessments where (hour(started_at) == 12)').show()

+-------------+
|started_at_12|
+-------------+
|          136|
+-------------+



##### Some exploratory analysis

In [17]:
spark.sql('select avg(keen_created_at) from assessments').show()

+------------------------------------+
|avg(CAST(keen_created_at AS DOUBLE))|
+------------------------------------+
|                1.5140290115508485E9|
+------------------------------------+



In [18]:
spark.sql('select * from ct').show()

+-------+-----+
|correct|total|
+-------+-----+
|      2|    4|
|      1|    4|
|      3|    4|
|      2|    4|
|      3|    4|
|      5|    5|
|      1|    1|
|      5|    5|
|      4|    4|
|      0|    5|
|      3|    4|
|      1|    1|
|      4|    6|
|      4|    6|
|      4|    5|
|      3|    4|
|      3|    4|
|      4|    4|
|      2|    4|
|      6|    6|
+-------+-----+
only showing top 20 rows



In [19]:
spark.sql('select * from questions').show()

+--------------------+--------------------+--------+
|                  id|             keen_id|my_count|
+--------------------+--------------------+--------+
|7a2ed6d3-f492-49b...|5a6745820eb8ab000...|       1|
|bbed4358-999d-446...|5a6745820eb8ab000...|       2|
|e6ad8644-96b1-461...|5a6745820eb8ab000...|       3|
|95194331-ac43-454...|5a6745820eb8ab000...|       4|
|95194331-ac43-454...|5a674541ab6b0a000...|       1|
|bbed4358-999d-446...|5a674541ab6b0a000...|       2|
|e6ad8644-96b1-461...|5a674541ab6b0a000...|       3|
|7a2ed6d3-f492-49b...|5a674541ab6b0a000...|       4|
|b9ff2e88-cf9d-4bd...|5a67999d3ed3e3000...|       1|
|bec23e7b-4870-49f...|5a67999d3ed3e3000...|       2|
|1ba75b31-64a4-4bd...|5a67999d3ed3e3000...|       3|
|1f7c5def-904b-483...|5a67999d3ed3e3000...|       4|
|1f7c5def-904b-483...|5a6799694fc7c7000...|       1|
|bec23e7b-4870-49f...|5a6799694fc7c7000...|       2|
|1ba75b31-64a4-4bd...|5a6799694fc7c7000...|       3|
|b9ff2e88-cf9d-4bd...|5a6799694fc7c7000...|   

In [20]:
spark.sql('select * from sequences').show()

+--------------------+--------------------+
|             keen_id|        sequences_id|
+--------------------+--------------------+
|5a6745820eb8ab000...|5b28a462-7a3b-42e...|
|5a674541ab6b0a000...|5b28a462-7a3b-42e...|
|5a67999d3ed3e3000...|b370a3aa-bf9e-4c1...|
|5a6799694fc7c7000...|b370a3aa-bf9e-4c1...|
|5a6791e824fccd000...|04a192c1-4f5c-4ac...|
|5a67a0b6852c2a000...|e7110aed-0d08-4cb...|
|5a67b627cc80e6000...|5251db24-2a6e-424...|
|5a67ac8cb0a5f4000...|066b5326-e547-4da...|
|5a67a9ba060087000...|8ac691f8-8c1a-403...|
|5a67ac54411aed000...|066b5326-e547-4da...|
|5a67ad9b2ff312000...|083844c5-772f-48d...|
|5a67b610baff90000...|5251db24-2a6e-424...|
|5a67ac9837b82b000...|b68128a9-6b50-41f...|
|5a67aaa4f21cc2000...|67457eec-4cad-416...|
|5a67ac46f7bce8000...|066b5326-e547-4da...|
|5a67aedaf34e85000...|7b754bca-91a1-4aa...|
|5a67aefef5e149000...|7b754bca-91a1-4aa...|
|5a67ae3f0c5f48000...|42a1e4c5-7a08-469...|
|5a67ad579d5057000...|d51a016b-0122-452...|
|5a67aae6753fd6000...|67457eec-4

##### 2. How many different exams are there?

In [25]:
spark.sql('select count(distinct(exam_name)) as distinct_exams from assessments').show()

+--------------+
|distinct_exams|
+--------------+
|           103|
+--------------+



In [26]:
extracted_assessments = assessments.rdd.map(lambda x: Row(**json.loads(x.value))).toDF()
#extracted_assessments.show()

### Linux Commands:

You would need to include at a minimum the following linux commands. If you use a markdown file, you can simply include them.  If you use a jupyter notebook, you can create a markdown cell for each one of these.  You can include other commands in addition to the minimum to help explain what you did.

##### 1) command to bring up the cluster

docker-compose up -d

docker-compose ps

docker ps -a

##### 2) command to create the kafka topic with a meaningful name (most students choose assessments)

docker-compose exec kafka kafka-topics --create --topic assessments --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

##### 3) command to publish the assessments json data to the kafka topic using kafkacat

docker-compose exec mids bash -c "cat /w205/project-2-kt-cason/assessment-attempts-20180128-121051-nested.json | jq '.[]' -c | kafkacat -P -b kafka:29092 -t assessments"

##### 4) command to shutdown the cluster

docker-compose down

docker-compose ps

docker ps -a